## **Dataset**

In [ ]:
from datasets import load_dataset

# Load IMDb dataset
dataset = load_dataset("imdb")

In [ ]:
dataset["train"][0]

{'text': 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [ ]:
# With small dataset: 50% full data
from datasets import load_dataset, concatenate_datasets

# Load full dataset
dataset = load_dataset("imdb")

# Tách riêng nhãn
train_dataset = dataset["train"]
positive = train_dataset.filter(lambda x: x["label"] == 1)
negative = train_dataset.filter(lambda x: x["label"] == 0)

# Lấy 50% (balanced): mỗi lớp lấy 12500 * 0.5 = 6250 mẫu
subset_pos = positive.shuffle(seed=42).select(range(6250))
subset_neg = negative.shuffle(seed=42).select(range(6250))

# Gộp lại và shuffle toàn bộ
balanced_subset = concatenate_datasets([subset_pos, subset_neg]).shuffle(seed=42)

# Thay vào tập train
dataset["train"] = balanced_subset


In [ ]:
dataset

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 12500
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['text', 'label'],
        num_rows: 50000
    })
})

## **Tokenizer**

In [ ]:
from transformers import AutoTokenizer

model_name = "thainq107/gpt-small-c4"
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [ ]:
# Tokenize function
def tokenize(example):
    return tokenizer(
        example["text"], padding="max_length", truncation=True, max_length=256
    )

# Tokenize full dataset
tokenized_ds = dataset.map(tokenize, batched=True)
tokenized_ds = tokenized_ds.remove_columns(["text"])
tokenized_ds.set_format("torch")

In [ ]:
tokenizer.pad_token_id

1

In [ ]:
tokenized_ds

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 25000
    })
    unsupervised: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 50000
    })
})

## **Model**

In [ ]:
from transformers import AutoConfig, AutoModelForSequenceClassification

# Load config từ model pretrained (không load weight!)
config = AutoConfig.from_pretrained("thainq107/gpt-small-c4")
config.num_labels = 2
config.id2label = id2label = {0: 'Negative', 1: 'Positive'}
config.label2id = {'Negative': 0, 'Positive': 1}

# Khởi tạo model mới từ config (với weight ngẫu nhiên)
model = AutoModelForSequenceClassification.from_config(config)

# Thiết lập pad_token_id nếu cần
model.config.pad_token_id = tokenizer.pad_token_id


In [ ]:
from transformers import AutoModelForSequenceClassification

id2label = {0: 'Negative', 1: 'Positive'}
label2id = {'Negative': 0, 'Positive': 1}

model = AutoModelForSequenceClassification.from_pretrained(
    "thainq107/gpt-small-c4",
    num_labels=2, id2label=id2label, label2id=label2id
)
model.config.pad_token_id = tokenizer.pad_token_id

Some weights of GPT2ForSequenceClassification were not initialized from the model checkpoint at thainq107/gpt-small-c4 and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
model

GPT2ForSequenceClassification(
  (transformer): GPT2Model(
    (wte): Embedding(50257, 512)
    (wpe): Embedding(512, 512)
    (drop): Dropout(p=0.1, inplace=False)
    (h): ModuleList(
      (0-5): 6 x GPT2Block(
        (ln_1): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (attn): GPT2Attention(
          (c_attn): Conv1D(nf=1536, nx=512)
          (c_proj): Conv1D(nf=512, nx=512)
          (attn_dropout): Dropout(p=0.1, inplace=False)
          (resid_dropout): Dropout(p=0.1, inplace=False)
        )
        (ln_2): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
        (mlp): GPT2MLP(
          (c_fc): Conv1D(nf=2048, nx=512)
          (c_proj): Conv1D(nf=512, nx=2048)
          (act): NewGELUActivation()
          (dropout): Dropout(p=0.1, inplace=False)
        )
      )
    )
    (ln_f): LayerNorm((512,), eps=1e-05, elementwise_affine=True)
  )
  (score): Linear(in_features=512, out_features=2, bias=False)
)

## **Training**

In [ ]:
import evaluate

accuracy = evaluate.load("accuracy")

def compute_metrics(eval_pred):
    logits, labels = eval_pred
    preds = logits.argmax(axis=-1)
    return accuracy.compute(predictions=preds, references=labels)

Using the latest cached version of the module from /home/s24thai/.cache/huggingface/modules/evaluate_modules/metrics/evaluate-metric--accuracy/f887c0aab52c2d38e1f8a215681126379eca617f96c447638f751434e8e65b14 (last modified on Wed Feb  5 15:33:17 2025) since it couldn't be found locally at evaluate-metric--accuracy, or remotely on the Hugging Face Hub.


In [ ]:
from transformers import TrainingArguments, Trainer

training_args = TrainingArguments(
    output_dir="imdb2-small-gpt2-small",
    eval_strategy="epoch",
    save_strategy="epoch",
    logging_strategy="epoch",
    per_device_train_batch_size=64,
    per_device_eval_batch_size=64,
    num_train_epochs=10,
    weight_decay=0.01,
    load_best_model_at_end=True,
    metric_for_best_model="accuracy",
    greater_is_better=True,
    save_total_limit=1,
    fp16=True,
)


In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_ds["train"],
    eval_dataset=tokenized_ds["test"],
    processing_class=tokenizer,
    compute_metrics=compute_metrics,
)

trainer.train()

Epoch,Training Loss,Validation Loss,Accuracy
1,0.420200,0.332835,0.855400
2,0.256300,0.410367,0.841720
3,0.154500,0.454666,0.846360
4,0.089300,0.655734,0.834960
5,0.057200,0.814169,0.828040
6,0.033100,1.457752,0.812400
7,0.020100,1.353611,0.811960
8,0.014800,1.315395,0.827000
9,0.007800,1.438292,0.828880
10,0.004700,1.453643,0.828040


TrainOutput(global_step=3910, training_loss=0.10581434989524315, metrics={'train_runtime': 652.7085, 'train_samples_per_second': 383.019, 'train_steps_per_second': 5.99, 'total_flos': 7263879168000000.0, 'train_loss': 0.10581434989524315, 'epoch': 10.0})